# Data Science - Toxikologische Vorhersagen


---
### Lernziele


- Sie können einen SVM trainieren.
- Sie verstehen den Grund für die Notwenigkeit Variablen zu skalieren.
- Sie können ein Random Forest Modell trainieren.
- Sie verstehen Y-Scrambling und die Notwendigkeit von Train/Test-Splits.
- Sie können Daten in eine Trainings- und eine Testsatz aufteilen.
---

Heute werden Sie einige Grundlagen der Data Science und des maschinellen Lernens lernen. Diese werden später für das trainieren von neuronaler Netze relevant sein. 
Als Beispiel werden wir Modelle erstellen, die die toxikologische Bedenklichkeit von Molekülen erkennen sollen.
Im konkreten Beispiel geht es um die Messung des **mitochondrialen Membranpotenzials** (MMP). Dieses wird als Indikator für die allgemeine Gesundheit der Zelle verwendet \[1\].


<img src="https://www.researchgate.net/publication/326685180/figure/fig4/AS:654070805172233@1532954040230/assay-of-a549-cells-mitochondrial-membrane-potential-with-Jc-1-staining-method-Notes.png" width=400 height=200 />

<center>Beispiel eines MMP Assays.<br> <i> Source: Liao et al.[2] licensed under CC-BY-NC</i></center>


Die Daten stammen aus dem Datensatz der Tox21 Challenge. Bei diesem Wettbewerb ging es darum, die toxikologischen Eigenschaften von Molekülen vorherzusagen. Zu diesem Zweck wurden die Messungen für insgesamt zwölf Assays zur Verfügung gestellt. Wir konzentrieren uns vorerst nur auf einen Assay und verwenden auch nicht den gesamten Datensatz, sondern nur etwa 2000 Moleküle. 


##### Zuvor aber werden wir anhand eines Beispiels den Effekt von Variablen Scaling betrachten.

---
**Referenzen:**
<br>
<br>
\[1\] Sakamuru, S., Attene-Ramos, M. S., & Xia, M. (2016). Mitochondrial membrane potential assay. In High-throughput screening assays in toxicology (pp. 17-22). Humana Press, New York, NY. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5375165/
<br>
<br>
\[2\] Liao, C., Xu, D., Liu, X., Fang, Y., Yi, J., Li, X., & Guo, B. (2018). Iridium (III) complex-loaded liposomes as a drug delivery system for lung cancer through mitochondrial dysfunction. International Journal of Nanomedicine, 13, 4417.

In [ ]:
import pandas as pd 
import numpy as np
!pip install rdkit==2022.3.4
!pip install seaborn
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import AllChem as Chem
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from rdkit.Chem.Lipinski import * 
from rdkit.Chem.rdMolDescriptors import CalcExactMolWt, CalcTPSA
from rdkit.Chem.Crippen import MolLogP, MolMR
from sklearn.preprocessing import MinMaxScaler
import seaborn as  sns
from matplotlib import pyplot as plt
import sys
%matplotlib inline
if 'google.colab' in sys.modules:
  !wget https://raw.githubusercontent.com/kochgroup/intro_pharma_ai/main/utils/utils.py
  %run utils.py
else:
  %run ../utils/utils.py # lädt vorgeschriebene Funktionen

# Scaling

Die Skalierung von Variablen kann für den Erfolg von Machine Learning Modellen entscheidend sein. Skalierung von Variablen bedeutet, dass wir die Skala der Werte einer Variablen ändern. Genauer gesagt wollen wir, dass alle Variablen die gleiche Skala verwenden. 

Beispielsweise liegt der Wert eines Hauses in € oft zwischen 50.000 und mehreren Millionen. Die Fläche des Hauses beträgt jedoch wahrscheinlich nur zwischen 10 und mehreren hundert Quadratmetern. Die Werte, die der Preis annehmen kann, sind viel größer als die der Fläche. 

Für einige Algorithmen, die auf Entfernungen oder Gradienten beruhen, kann dies ein Problem darstellen, da die Skala der Variablen für diese Variable einen direkten Einfluss auf die Bedeutung der Variable hat.

Variablen mit einer größeren Skala wird mehr Bedeutung beigemessen, auch wenn die Skala willkürlich ist. Man könnte auch den Preis eines Hauses in 1000€ ausdrücken (aus 50.000 wird 50). Die Skala ändert sich, aber nicht die eigentliche Variable.

Um diesen Effekt zu vermeiden, skalieren wir Variablen auf einheitliche Größen.
Zum Beispiel skaliert das **MinMax-Scaling** alle Werte zwischen '0' und '1'. 

Im folgenden Beispiel sehen Sie, welchen Einfluss die Skalierung auf eine Support Vector Machine haben kann.

Zunächst laden Sie die Daten mit Python ein. 

___
**Dateitypen:**
<br> <br>
Es gibt viele verschiedene Formate, in denen Daten gespeichert werden. Das wohl am häufigsten verwendete ist das Format `comma separated value`. Dieses erkennt man an der Abkürzung `.csv` am Ende einer Datei.  <div style="float: right;"><img  src="https://images.freeimages.com/images/large-previews/7f6/tab-key-1243535.jpg" width=150 height=100 /><center>freeimages.com: T. Al Nakib</center></div>

Wie der Name schon vermuten lässt, werden die einzelnen Werte durch ein Komma getrennt. Werte, die in dieselbe Zeile gehören, werden in dieselbe Zeile geschrieben und durch das Komma getrennt. Andere Dateiformate, die häufig verwendet werden, sind Textdateien `.txt`. Hier kann man nicht direkt aus der Endung schließen, wie die Struktur aufgebaut ist.
Oft wird aber das System "Werte in einer Zeile gehören in eine Zeile" eingehalten. Lediglich das Zeichen, das einzelne Werte trennt, kann sich unterscheiden. Ein häufig verwendetes Trennzeichen ist der Tab(ulator). Das Tabulatorzeichen wird auch oft im `.smi` Format verwendet. Sie werden dieses Format häufiger sehen, wenn Sie mit SMILES arbeiten. Wenn Sie sich nicht sicher sind, welcher "seperator" verwendet wird, können Sie die Datei mit einem einfachen Texteditor öffnen. Unter Windows zum Beispiel "Notepad". Hier sollten Sie sehen, wie die Werte voneinander getrennt sind.

In diesem Fall werden die Daten im Format `.tab` gespeichert. Die Werte werden also durch einen Tab getrennt. Sie können die Funktion `pd.read_csv()` auch dann verwenden, wenn die Datei keine `.csv`-Datei ist. 
Aber dann müssen Sie zusätzlich den `Seperator` `sep= "\t"` angeben. Das Symbol `"\t"` wird dann als Tab erkannt.

In [ ]:
print("ab")
print("a\tb")

Wie Sie vielleicht schon von einigen der geladenen Libraries gesehen haben, werden wir heute einige Funktionen verwenden, die Ihnen noch unbekannt sind. 

- `pandas` enthält nützliche Funktionen für die Verarbeitung großer Datenmengen und kann so ziemlich alles, was Excel auch kann (einiges sogar besser). `pandas` erstellt sogenannte `DataFrames`. DataFrames sind ähnlich zu 2D `arrays` in `numpy` und speichern Daten in Zeilen und Spalten. Der Unterschied ist, dass wir verschiedene Arten von Daten in einem `DataFrame` speichern können. Zum Beispiel `floats` und `strings` in zwei verschiedenen Spalten. Wir können den Spalten und Zeilen auch Namen zuweisen, um einen besseren Überblick über unsere Daten zu haben.

- Sie kennen `sklearn` bereits aus dem ROC-AUC (Woche 04).  `sklearn` bringt viele Funktionen mit, die für die Aufbereitung von Daten wichtig sind. Außerdem gibt es in `sklearn` mehrere maschinelle Lernalgorithmen, darunter den Random Forest und Support Vector Machines.

In der folgenden Zelle werden die Daten für ein einfaches Beispiel eingelesen. 

In [ ]:
toy_beispiel = pd.read_csv("https://uni-muenster.sciebo.de/s/XTC29EBCPfMfqqh/download", sep = "\t")

print("Type:", type(toy_beispiel))
print("Shape:",toy_beispiel.shape)
toy_beispiel.head()


Die mit `pandas` eingelesenen Daten werden zunächst in einem `DataFrame` gespeichert. Dies ist eine Tabelle mit Zeilen und Spalten. Mit `.head()` können Sie die ersten 5 Zeilen eines `DataFrame` anzeigen. Die Daten enthalten drei Spalten: `x1` `x2` und `y`. Insgesamt enthält die Datei 150 Einträge. Also insgesamt 150 Messpunkte. 

`y` gibt die fiktive Zugehörigkeit der einzelnen Datenpunkte zu einer von zwei Klassen an.

Um eine bessere Übersicht zu bekommen, erstellen wir ein Diagramm für die Daten.

In [ ]:
plt.plot(toy_beispiel.x1, toy_beispiel.x2,"o")
plt.plot( toy_beispiel.x1[toy_beispiel.y==1],  toy_beispiel.x2[toy_beispiel.y==1],"o")
plt.xlabel("x1")
plt.ylabel("x2")

Wie Sie sehen können, wird eine Klasse von der anderen "eingeschlossen". Wir wollen eine SVM trainieren, um die beiden Klassen zu unterscheiden.

Sie sollten auch feststellen, dass sich die Skalen der beiden Eingangsvariablen `x1` und `x2` deutlich unterscheiden. 
 

`x1` Werte liegen ungefähr zwischen `-1` und `1`. <br>`x2` Werte befinden sich zwischen ca. `8` und `12`.

Wir werden die Daten zunächst nicht skalieren und direkt eine SVM auf diese Daten trainieren. Hierfür bietet das Modul `sklearn.svm` einige Funktionalitäten. Wie bei der linearen Regression wird mit `sklearn` zunächst das Modell erstellt und dann trainiert.
Zur Klassifizierung kann die Funktion `SVC` (Support Vector Classification) verwendet werden. Zuerst erstellen wir eine Variable `model`. Diese enthält den Support Vector Classifier (`SVC`). Um ihn zu trainieren, verwenden wir die Funktion `.fit(x,y)`, um den Klassifikator an unsere Daten anzupassen. 

Bis jetzt haben wir nur den SVC trainiert, um seine Vorhersagen zu erhalten, müssen wir wieder die Funktion `model.predict(x)` verwenden.


**Wichtig:** Sie haben es vielleicht oben schon gesehen, in `pandas` können wir Variablen (d.h. Spalten) direkt aus dem `DataFrame` auswählen. So selektiert `toy_beispiel.y` die Spalte `y` aus dem DataFrame `toy_beispiel`. 
Wenn Sie Werte mit der klassischen Indizierung auswählen wollen, wie bei `arrays`, müssen Sie zuerst ein `.iloc[]` an den Namen des DataFrame anhängen. Die Indizierung funktioniert dann genauso wie bei `numpy`.

`toy_beispiel.iloc[:,:2]` wählt die ersten beiden Spalten aus, also `x1` und `x2`.

In [ ]:
from sklearn.svm import SVC

model = SVC()
model.fit(toy_beispiel.iloc[:,:2], toy_beispiel.y)
y_pred = model.predict(toy_beispiel.iloc[:,:2])
y_pred

Um die Güte der Vorhersagen zu beurteilen, können Sie wieder die Accuracy/Genauigkeit verwenden. Dazu können Sie die gleiche Funktion wie in der letzten Woche verwenden. Berechnen Sie die Accuracy für die Vorhersagen dieses Modells. 

In [ ]:
def accuracy(y_true, y_pred):
    return np.sum(y_true==y_pred)/len(y_true)

accuracy(_____, ____)

<details>
    <summary><b>Lösung:</b></summary>

```python
accuracy(toy_beispiel.y, y_pred)
```
</details>

Die Accuracy liegt bei etwa `0.7`. Nicht schlecht, aber es gibt noch Raum für Verbesserungen. Mit einer der vorgeschriebenen Funktionen in `utils.py` können Sie auch die Entscheidungsgrenzen sichtbar machen. 

In [ ]:
plot_svc(toy_beispiel, model)

Sie können sehen, dass die Entscheidungsgrenze sehr lang ist. Das liegt daran, dass die Skala von `x2` größer ist. Das bedeutet, dass der Abstand von der Entscheidungsgrenze zu den Datenpunkten zweier Klassen (der maximiert werden muss) für `x2` leichter zu maximieren ist als für `1`. Deshalb sehen wir eine gute Entscheidungsgrenze für die Werte von `x2`, aber nicht für `x1`. 

Um dies zu ändern, können wir versuchen, die Daten zu skalieren.
Dazu verwenden wir den so genannten `MinMax`-Skalierer, bei dem alle Werte zwischen `0` und `1` skaliert werden. Wir wenden diesen Skalierer auf beide Eingabevariablen (`x1`, `x2`) an. 

In [ ]:
def min_max(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

toy_beispiel.x1 = min_max(toy_beispiel.x1)
toy_beispiel.x2 = min_max(toy_beispiel.x2)

In [ ]:
plt.plot(toy_beispiel.x1, toy_beispiel.x2,"o")
plt.plot( toy_beispiel.x1[toy_beispiel.y==1],  toy_beispiel.x2[toy_beispiel.y==1],"o")
plt.xlabel("x1")
plt.ylabel("x2")

Der Plot sieht genau so aus wie der früher, aber die Skalen, also die x- und y-Achse, haben sich geändert. 
Das heißt, die relative Beziehung der Werte hat sich nicht geändert.
Können Sie nun ein `model_2` mit `SVC` erstellen, das mit den skalierten Werten trainiert wird? Berechnen Sie auch die Accuracy!

In [ ]:
model_2 = ____
model_2.fit(_____, ______)
y_pred = model_2.predict(toy_beispiel._____)
accuracy(______,_____)

<details>
    <summary><b>Lösung:</b></summary>

```python
model_2 = SVC()
model_2.fit(toy_beispiel.iloc[:,:2], toy_beispiel.y)
y_pred = model_2.predict(toy_beispiel.iloc[:,:2])
accuracy(toy_beispiel.y, y_pred)
```
</details>

Allein durch die Skalierung der Inputvariablen ergibt sich eine Verbesserung der Genauigkeit um 0,3.
Eine deutliche Verbesserung ist auch anhand der Entscheidungsgrenze zu erkennen.

In [ ]:
plot_svc(toy_beispiel, model_2)

Auch wenn die Daten nur für dieses Beispiel generiert wurden, zeigt es doch, warum die Skalierung der Inputvariablen so wichtig ist.

# Train & Test Split


Von einem theoretischen Beispiel kommen wir nun zu einer praktischen Aufgabe: einer toxikologischen Vorhersage.
Die ursprünglich erwähnten Assay Daten können mit `pd.read_csv()` direkt aus dem Internet geladen werden.

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/filipsPL/tox21_dataset/master/compounds/sr-mmp.tab", sep = "\t")

print("Shape:",data.shape)
data.head()

Die Daten enthalten drei Spalten: `Compound`, `SMILES` und `activity`. Insgesamt enthält die Datei 2246 Moleküle.

- `Compound` enthält die ID, mit der die Werte im ursprünglichen Datensatz unterschieden werden können.
- `SMILES` enthält die SMILES-`strings`.
- `activity` enthält die Ergebnisse des Assays. Ein Molekül ist aktiv (`1`) oder inaktiv (`0`).

---
Wenn Sie wissen wollen, wie viele aktive und damit giftige Moleküle in dem Datensatz enthalten sind, können Sie einfach die Summe der Spalte `activity` berechnen.

In [ ]:
np.sum(data.activity) # data.activity wählt die in 'data' enthalten Spalte 'activity' aus

Für die Analyse ist die `Compounds` Spalte nicht wichtig. Darum wird sie aus `data` entfernt. Danach benennen wir noch die Spalten neu, damit alle Namen kleingeschrieben werden.

In [ ]:
data = data.iloc[:,1:] #alle Spalten bis auf die erste (index 0) werden ausgewählt
data.columns = ["smiles", "activity"]
data.head()

Sie haben Daten, aber noch keinen Input für ihr Modell. SMILES sind `str`-Variablen, aber Modelle benötigen numerische Variablen als Eingabe. Das bedeutet, dass Sie noch so genannte Features erstellen müssen. Hierfür können Sie die Deskriptoren aus dem "Cheminformatics"-Notebook verwenden. In den folgenden Zellen wandeln wir die SMILES in `mol` um und verwenden sie zur Berechnung der Deskriptoren. 

Diesmal berechnen wir auch die molare Refraktivität (Maß für die Gesamtpolarisierbarkeit), die Anzahl der drehbaren Bindungen und die TPSA (topologische polare Oberfläche). Dann kombinieren wir alle Variablen in einem `DataFrame`.

Passen Sie die `for-loops` an:

In [ ]:
# Wir konvertieren all SMILES zu mols
mols = np.array([Chem.MolFromSmiles(x) for x in ________ ]) # Welche SMILES müssen ausgewählt werden

# 1) N hydrogen bond donors
num_hb_donors = [NumHDonors(x) for x in _______ ] # durch welche Variable loopen wir

# 2) Hydrogen bond acceptors
num_hb_acceptors = [NumHAcceptors(x) for _____ in ______] # durch welche Variable loopen wir

# 3) Number of rotable bonds 
num_rotablebonds = [NumRotatableBonds(x) for x in mols]

# 4) Molecular Mass: CalcExactMolWt()
mw = [ _________(___ ) for x in mols]  # Berechnen Sie die das Gewicht mit CalcExactMolWt()

# 5) log P: MolLogP()
logP = [________ ___ __ __ ______] # Berechnen Sie den logP mit MolLogP()

# 6) Molar refractivity 
mr = [MolMR(x) for x in mols]

# 7) Polar Surface
tpsa = [CalcTPSA(x) for x in mols]

aux_data=pd.DataFrame({
    "hb_donors": num_hb_donors,
    "hb_acceptors": num_hb_acceptors,
    "rotable_bonds": num_rotablebonds,
    "mw": mw,
    "logP": logP,
    "mr":mr,
    "tpsa":tpsa
    })

aux_data["activity"] = data.activity # Wir fügen noch die activity hinzu
aux_data


<details>
    <summary><b>Lösung:</b></summary>

```python
# Wir konvertieren all Smiles zu mols
mols = np.array([Chem.MolFromSmiles(x) for x in data.smiles])

# 1) N hydrogen bond donors
num_hb_donors = [NumHDonors(x) for x in mols]

# 2) Hydrogen bond acceptors
num_hb_acceptors = [NumHAcceptors(x) for x in mols]

# 3) Number of rotable bonds 
num_rotablebonds = [NumRotatableBonds(x) for x in mols]

# 4) Molecular Mass
mw = [CalcExactMolWt(x) for x in mols]

# 5) log P
logP = [MolLogP(x) for x in mols]

# 6) Molar refractivity 
mr = [MolMR(x) for x in mols]

# 7) Polar Surface
tpsa = [CalcTPSA(x) for x in mols]

aux_data=pd.DataFrame({
    "hb_donors": num_hb_donors,
    "hb_acceptors": num_hb_acceptors,
    "rotable_bonds": num_rotablebonds,
    "mw": mw,
    "logP": logP,
    "mr":mr,
    "tpsa":tpsa
    })

aux_data["activity"] = data.activity # Wir fügen noch die activity hinzu
aux_data
```
</details>

Der `DataFrame` `aux_data` hat für jeden Deskriptor eine eigene Spalte, also insgesamt sieben. Die erste Zeile enthält die Deskriptoren für das erste Molekül, die zweite für das zweite, und so weiter.
Sie können nun diesen Datensatz verwenden, um ein Modell zu erstellen. 
Es sind jedoch einige zusätzliche Schritte erforderlich, bevor Sie ein funktionierendes Modell erstellen und Vorhersagen treffen können.

Zunächst haben wir zwei Arten von Variablen. Variablen wie die Anzahl der drehbaren Bindungen werden als "diskret" bezeichnet, weil sie nur ganze Zahlen enthalten. Es gibt keine `3,5` drehbaren Bindungen in einem Molekül. Im Gegensatz dazu sind Variablen wie logP oder TPSA "kontinuierlich", d.h. Variablen, die Werte über den gesamten Zahlenbereich annehmen können.

Darüber hinaus haben wir Variablen mit unterschiedlichen Skalen. Die Werte für das Gewicht sind viel größer als beispielsweise die Werte für logP oder die Anzahl der HB-Akzeptoren. 

Da wir jedoch als Nächstes ein Random-Forest-Modell verwenden wollen, müssen die Variablen nicht skaliert werden.
Das liegt daran, dass Random Forests keine Abstände oder Gradienten verwenden.

In [ ]:
# Zunächst teilen wir den Datensatz in 'x' und 'y' also input und output
x = aux_data.iloc[:,:7].values #'[:,:7]' Alle Spalten bis auf die 7 Spalte
y = aux_data.iloc[:,7].values #'[:,7]' Nur Spalte 7

Die neue Variable `x` ist jetzt ein `np.array` (anstelle eines `DataFrame`). Das wurde durch die Endung `.values` möglich gemacht. So kann mann schnell `pd.DataFrame` zu `np.arrays` konvertieren. 

Sie können nun ein Random Forest Modell trainieren. Ähnlich wie bei `SVC` müssen Sie zuerst eine Variable mit dem Modell erstellen und dann `.fit()` verwenden, um das Modell auf den Daten zu trainieren.

In [ ]:
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42) 
# n_estimators wählt aus wie viele Trees benutzt werden sollen
rf.fit(x, y)

Um zu sehen, wie gut das Modell funktioniert, müssen Sie noch die Vorhersagen des Modells für unsere Daten extrahieren. Anders als üblich, verwenden wir dafür die Funktion <br>`.predict_proba(x)[:,1]`. Mit dem trainierten `rf` Modell machen wir die Vorhersagen für `y_hat`.

In [ ]:
y_hat=rf.predict_proba(x)[:,1]
y_hat

Die vorhergesagte Wahrscheinlichkeit für das erste Molekül ist `0.014`. Wie bei der logistischen Regression bedeutet dies, dass das Molekül nach unserem Modell in 1,4 % der Fälle bei dem MMP-Test aktiv ist, d. h. es ist unwahrscheinlich, dass es toxisch ist. Je höher die Wahrscheinlichkeit ist, desto wahrscheinlicher ist es (laut dem Modell), dass das Molekül im Assay aktiv ist. 
Häufig wird 0,5 als "Cut-off"-Wert gewählt. Bei einem Wert von 0,5 würden wir also davon ausgehen, dass das Modell diese Moleküle als toxisch einstuft. 

Um besser beurteilen zu können, wie gut das Modell funktioniert, vergleichen Sie die vorhergesagten Werte `pred_y` mit den tatsächlichen Werten `y`.

Dazu kann man wieder die Accuracy nehmen. Um die Accuracy zu berechnen, muss man zunächst die Wahrscheinlichkeiten auf `0` und `1` runden.

In [ ]:
y_pred = np.round(y_hat)
accuracy(y, y_pred)

Dann können wir den AUC mit der Funktion `roc_auc_score()` berechnen.
Anders als bei `accuracy` werden hier die Wahrscheinlichkeiten `y_hat` anstelle der gerundeten Werte `y_pred` verwendet.

In [ ]:
roc_auc_score(y, ____)

<details>
    <summary><b>Lösung:</b></summary>

```python
roc_auc_score(y,y_hat)
```
</details>

Sehr gut. Das Modell ist nahezu perfekt bei der Vorhersage. Es trifft in 99 % der Fälle die richtige Entscheidung. In der folgenden Zelle werden die falsch klassifizierten Moleküle ausgewählt und mit ihrer vorhergesagten Wahrscheinlichkeit angezeigt. Es ist nicht notwendig, dass Sie diesen Code verstehen. 

In [ ]:
falsch_klassifizierte=np.where(y_pred!=y)[0]
Draw.MolsToGridImage(mols[falsch_klassifizierte],
                     legends=["Vorhergesagte Wahrscheinlichkeit:\n"+str(np.round(x,3)) for x in y_hat[falsch_klassifizierte]],
                     useSVG=True)

Auffallend ist, dass die Wahrscheinlichkeiten für diese Moleküle meist relativ nahe bei 0,5 liegen. Das bedeutet, dass das Modell für diese Moleküle nicht sehr sicher war. Insgesamt wurden jedoch nur 19 Moleküle falsch klassifiziert, so dass wir uns nicht allzu viele Sorgen machen sollten.


## Y-Scrambling

##### Das Problem:

*Hat das Modell wirklich gelernt, was für den MMP-Assay wichtig ist. Oder hat sich das RF-Modell nur unsere Daten auswendig gelernt?*.

Wir können dies mit einem einfachen Test herausfinden. Wir trainieren das RF-Modell erneut, aber mischen wir die Variable `activity` zufällig. Das heißt, die echten Messungen werden gemischt und neu, zufällig auf die Moleküle verteilt. Dieser Vorgang wird auch als **Y-scrambling** bezeichnet. 

Nehmen wir an, unsere realen Daten sehen wie folgt aus:

smiles|Deskriptor 1| Deskriptor 2|activity
------|------------|-------------|--------
SMILES 1|$x_{1,1}$ |$x_{1,2}$|$y_1$
SMILES 2|$x_{2,1}$ |$x_{2,2}$|$y_2$
SMILES 3|$x_{3,1}$ |$x_{3,2}$|$y_3$
SMILES 4|$x_{4,1}$ |$x_{4,2}$|$y_4$

Für jedes SMILES wurden zwei Deskriptoren berechnet. Wir haben auch die Aktivität ($y_1$-$y_4$) für jedes Molekül aufgezeichnet. Die Aktivität $y_1$ ist die gemessene Aktivität von SMILES 1 und so weiter.

Nach dem *Y-Scrambling* sehen unsere Daten wie folgt aus:

smiles|Deskriptor 1| Deskriptor 2|activity
------|------------|-------------|--------
SMILES 1|$x_{1,1}$ |$x_{1,2}$|$y_2$
SMILES 2|$x_{2,1}$ |$x_{2,2}$|$y_3$
SMILES 3|$x_{3,1}$ |$x_{3,2}$|$y_4$
SMILES 4|$x_{4,1}$ |$x_{4,2}$|$y_1$

Die $y$ Werte wurden zufällig anderen Molekülen zugeordnet.

Das Y-Scrambling führt zum Verlust der tatsächlichen Beziehungen, die zwischen den Variablen `x` (d.h. unseren Deskriptoren wie logP,...) und der zu vorhersagenden Variable `y` bestehen.  Das liegt daran, dass die Beziehung jetzt nur noch zufällig ist. Wenn unser Random Forest tatsächlich Muster lernt, anstatt die Daten auswendig zu lernen, dann sollte das Modell bei diesem Datensatz schlechter abschneiden.

Um dies auszuprobieren, benötigen Sie die Bibliothek `random`. Die Funktion `random.shuffle()` sortiert die Werte von `y` zufällig neu. Sie können dann das Random-Forest Modell erneut trainieren.

In [ ]:
import random
random.seed(15) #ein Seed stell sicher das Sie alle die selbe zufälligen Daten erhalten
y_random=np.array(y) # erst speichern wir y in einem array
random.shuffle(y_random) # dann shuffeln wir y_random, wir müssen diese Variable nicht extra speichern

Wir haben gerade die Aktivitätsinformationen neu gemischt. Jetzt können wir ein Random-Forest Modell trainieren. Dieses Mal verwenden wir jedoch nicht `y`, sondern `y_random`.

In [ ]:
# Das Modell wird neu trainiert
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)
rf.fit(x, ______) # Welcher y Variable brauchen Sie?
y_hat=rf.predict_proba(x)[:,1]

<details>
    <summary><b>Lösung:</b></summary>

```python
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)
rf.fit(x, y_random) # Welcher y Variable brauchen Sie
y_hat=rf.predict_proba(x)[:,1]
```
</details>




Jetzt berechnen Sie erneut die Accuracy (achten Sie darauf, dass wir wieder `y_random` und nicht `y` für `y_true` benutzen):

In [ ]:
y_pred = np.round(y_hat)
accuracy(y_random, y_pred)

In der Tat verschlechtert sich die Accuracy nach dem Y-Scrambling. Aber die Accuracy liegt immer noch bei über 90 %. Das Random-Forest-Modell ist immer noch relativ gut bei der Vorhersage von toxikologischer Bedenklichkeit, auch wenn die von Ihnen verwendeten Daten überhaupt keinen Sinn mehr machen. Das Modell hätte überhaupt nicht mehr lernen können, da kein Zusammenhang zwischen `x` und `y` existiert. Das RF-Modell hat seine Accuracy also nur durch Auswendiglernen erreicht. 

Aus diesem Grund wird immer ein Testdatensatz verwendent. Dieser Testdatensatz wird beim Training nicht verwendet, und so sieht das Modell diese Moleküle zum ersten Mal, wenn wir die Qualität des Modell evaluieren wollen. Das Auswendiglernen der Moleküle im Trainingsdatensatz kann immer noch passieren, hilft dem Modell aber nicht bei dem Molekülen die wir im Testdatensatz haben.  

---
Häufig werden Datensätze nicht nur in Trainings-/Testsets, sondern in Trainings-/Validierungs-/Testsets aufgeteilt.
Die Modelle werden dann auf der Grundlage des Validierungssets optimiert und nur das optimierte Modell wird dann auf dem Testset getestet.
In der Chemieinformatik wird das Validierungsset manchmal auch als Testset bezeichnet. Das eigentliche Testset wird dann als externers Validierungsset bezeichnet.

---
Auch hier gibt es Funktionen, die Ihnen die Arbeit abnehmen. Die Funktion `train_test_split` teilt die Daten in ein Testset und eine Trainingsset auf. Wir verwenden 80 % des Datensatzes für das Training und den Rest für die Validierung. Die Moleküle werden dann nach dem Zufallsprinzip auf die beiden Sets aufgeteilt. Dann werden die Daten erneut in `x` und `y` aufgeteilt, aber diesmal für `train` und `test` getrennt.



In [ ]:
train, test=train_test_split(aux_data,test_size= 0.2, train_size= 0.8, random_state=1234)

train_x = train.iloc[:,:7]
train_y = train.iloc[:,7]
test_x = test.iloc[:,:7]
test_y =  test.iloc[:,7]
f"Train Shape: {train.shape}, Test Shape: {test.shape}"

Das Trainingsset enthält nur noch 1796 Moleküle und das Testset 450. 

Wir trainieren zunächst den Random Forest nur mit den Trainingsdaten:

In [ ]:
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)
rf.fit(train_x, train_y)

Die Vorhersagen machen wir aber nur für die Moleküle im Testset.

In [ ]:
y_hat=rf.predict_proba(test_x)[:,1]
y_pred = np.round(y_hat)

Wir können jetzt auch die Accuracy berechnen. Welche Variable brauchen wir nun für `y_true`?

In [ ]:
accuracy(___, y_pred)

<details>
    <summary><b>Lösung:</b></summary>

```python
accuracy(test_y, y_pred)
```
</details>

Die Genauigkeit hat sich um einiges verschlechtert, ist aber immer noch gut. Dieses Mal können wir jedoch sicher sein, dass die Leistung nicht auf das Auswendiglernen zurückzuführen ist, da das Modell diese Moleküle noch nie gesehen hat. Wenn wir nun den Y-Scrambling anwenden, sollte sich die Leistung drastisch verschlechtern. 
Wir ersetzen die `aux_data.activity` durch die zuvor erstellten `y_random`. Dies sind die gemischten `y`-Werte und wir wiederholen die Analyse.

In [ ]:
aux_data.activity = y_random 

Dann teilen wir die Daten erneunt in Trainings- und Testset ein.

In [ ]:
train_random, test_random=train_test_split(aux_data,test_size= 0.2, train_size= 0.8, random_state=1234)
train_x_random = train_random.iloc[:,:7]
train_y_random = train_random.iloc[:,7]
test_x_random = test_random.iloc[:,:7]
test_y_random =  test_random.iloc[:,7]

Wir wiederholen das Training mit den randomisierten Daten. Dann lassen wir das Modell Vorhersagen für das Testsset machen.

In [ ]:
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)
rf.fit(train_x_random, train_y_random)
y_hat=rf.predict_proba(test_x_random)[:,1]

Zum Schluss berechnen wir die Genauigkeit (Accuracy). 

In [ ]:
y_pred = np.round(y_hat)
accuracy(test_y_random, y_pred)

Mit dem Benutzen eines Testset fällt die Genauigkeit des Modells mit den Y-scrambled Daten drastisch auf ungefähr 50 %. Es ist nicht besser als ein Model, das einfach raten würde.
Nur durch die Verwendung eines Testsets konnten wir zeigen, dass das Modell etwas gelernt hat, das über das Auswendiglernen hinausgeht
Es ging vor allem darum, zu zeigen, wie wichtig es ist, ein Modell nicht anhand des Trainingssets zu bewerten. 

Y-Scrambling wird in der Praxis selten verwendet. Aber die OECD beispielsweise verlangt einen Y-Scrambling-Test zur Validierung von QSAR-Modellen (Quantitative Structure-Activity Relationship).

## Feature Importance

Als letzten Schritt wollen wir uns noch die **Feature Importance** anschauen. Die Feature Importance gibt an, wie wichtig die einzelnen Inputvariablen für die Entscheidung sind. Je nachdem, welchen ML Algorithmus Sie verwenden, können Sie die Feature Importance relativ leicht extrahieren. Wir trainieren unseren RF zunächst erneut, diesmal mit einem Datensplit.

In der nächsten Zelle wird zunächst das Random-Forest Modell erneut trainiert (ohne y-scrambling). Dannach können wir usn die Feature Importance von dem Modell ausgeben.

In [ ]:
aux_data.activity = data.activity

train, test=train_test_split(aux_data,test_size= 0.2, train_size= 0.8, random_state=1234)
train_x = train.iloc[:,:7]
train_y = train.iloc[:,7]
test_x = test.iloc[:,:7]
test_y =  test.iloc[:,7]


# Train Model
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)
rf.fit(train_x, train_y)
y_hat=rf.predict_proba(test_x)[:,1]


In [ ]:
feat_importances = pd.Series(rf.feature_importances_, index=aux_data.columns.values[:-1])
feat_importances.nlargest(20).nsmallest(20).plot(kind='barh')

Es wird deutlich ersichtlich, dass der LogP der wichtigste Parameter für die Bestimmung der Toxizität ist, während die Anzahl an H-Brücken-Donoren und -Akzeptoren weniger relevant sind. 

Das der LogP Wert wichtig ist, ist nicht überraschend. Wir zum Beispiel die Density Plots von aktiven und inaktive Molekülen anschauen.

In [ ]:
import seaborn as sns
sns.kdeplot(aux_data.logP[aux_data.activity==1], color="red")
sns.kdeplot(aux_data.logP[aux_data.activity==0])

Hier sehen wir einen deutlichen Trend. Bei höherem LogP ist das Molekül eher aktiv. 

Probieren Sie es selbst mit den anderen Deskriptoren aus(`hb_donors`, `hb_acceptors`, `rotable_bonds`, `mw`, `mr`, `tpsa`). Welche Deskriptoren haben neben dem LogP unterschiedliche Verteilung basierend auf der Aktivität?

# Übungsaufgabe 

Sie haben bereits Fingerabdrücke als Molekülrepräsentation kennengelernt. Da sie leicht zu berechnen sind und immer eine feste Länge haben, eignen sie sich gut als Input für ML Modelle. Allerdings sind Fingerabdrücke für den Menschen nicht so einfach zu interpretieren.

Ihre Aufgabe wird es sein, erneut ein Random Forest Modell zu trainieren. Diesmal werden Sie den ECFP4 als Input verwenden.

Für Sie wurde  die Funktion `get_fingerprints()` bereits vorgeschrieben. Mit ihr können Sie Fingerabdrücke aus den SMILES berechnen.

In [ ]:
fps = get_fingerprints(data)
fps["activity"] = data.activity
fps.head()

`fps` enthält insgesamt 2049 Spalten. 2048 davon sind die jeweiligen Bits des Fingerabdrucks. Die letzte Spalte enthält die `activity`.

Zunächst wird der Datensatz in `training` und `test` unterteilt. 80 % der Daten sollten im Trainingsset und 20 % im Testset enthalten sein.

In [ ]:
train, test = train_test_split(_____,test_size= ___ , train_size= ______, random_state=1234)

train_x = __________
train_y = __________
test_x = __________
test_y = __________ 

Nach der Aufteilung der Daten trainieren Sie einen Random Forest Classifier mit dem Trainingsdatensatz.
Anschließend verwenden Sie das trainierte Modell, um die Moleküle im Testdatensatz `test` zu klassifizieren.

In [ ]:
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)
rf.fit(_____, _____)
y_hat=rf.predict_proba(______)[:,1]

In [ ]:
roc_auc_score(___,____)

Wir können uns auch noch einmal die Feature Importance ansehen:

In [ ]:
feat_importances = pd.Series(rf.feature_importances_, index=range(2048))
feat_importances.nlargest(20).nsmallest(20).plot(kind='barh', title = "Importance of Features")

Leider lässt sich dieser Plot nicht mehr so gut interpretieren, auch wenn deutlich wird, dass die obersten fünf Bits für die Aktivitätsvorhersage wichtig sind.

Die Bits können auch nicht so gut dargestellt werden. Mit RDKit können wir aber die Substrukturen darstellen, die jeden Bit zugeordnet sind.

In [ ]:
most_important_bits = feat_importances.nlargest(20).index.values
print("Die 20 wichtigsten bits:", most_important_bits)
mol_ll = []
bi_ll = []


for i in range(20):
    bit = most_important_bits[i]
    for x in data.smiles:
        bi ={}
        mol = Chem.MolFromSmiles(x)
        fp = Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius=2, bitInfo=bi)
        if np.sum(np.array(list(bi))==bit)>0:
            mol_ll.append(mol)
            bi_ll.append(bi)
            break
        
prints=[(mol_ll[i],most_important_bits[i], bi_ll[i]) for i in range(20)]

Draw.DrawMorganBits(prints, useSVG=True, molsPerRow=3, legends= [str(most_important_bits[i]) for i in range(20)], subImgSize= [300,300])

Das wichtigste Bit ist für unsere Daten eine phenolische Hydroxygruppe (aromatische Atome sind gelb hervorgehoben, das zentrale Atom ist blau). Auch sonst sind viele aromatische Fragmente in den wichtigsten Bits vertreten.